# Solar power production model

The following notebook is a shortned version of a project to build a model that can predict solar energy production for every hour of the next day.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# Data
### Importing data

In [4]:
train_a = pd.read_parquet('data/A/train_targets.parquet')

In [5]:
X_train_estimated_a = pd.read_parquet('data/A/X_train_estimated.parquet')

In [6]:
X_train_observed_a = pd.read_parquet('data/A/X_train_observed.parquet')

In [7]:
X_test_estimated_a = pd.read_parquet('data/A/X_test_estimated.parquet')

# Feature Engineering

Scales ceiling height

In [8]:
scale = X_train_observed_a.loc['2020-03-25':, 'ceiling_height_agl:m'].max() / X_train_observed_a.loc['2019-07-01':'2020-03-25', 'ceiling_height_agl:m'].max()

X_train_observed_a.loc[:'2020-03-25', 'ceiling_height_agl:m'] *= scale

Is night feature

In [48]:
is_night_data = X_test_estimated_a.copy()
is_night_data['is_night'] = is_night_data['sun_elevation:d'].apply(lambda x: 1.0 if x < -0.5 else 0.0)

In [9]:
X_train_estimated_a.drop(columns=['date_calc'], inplace=True)

X_test_estimated_a.drop(columns=['date_calc'], inplace=True)

In [50]:
def convertDate(df, date_col):
    day_of_year = df[date_col].dt.dayofyear
    normalized_day = (day_of_year - 1) / 365
    df['day_curve'] = np.sin(2 * np.pi * normalized_day)
    hour = df[date_col].dt.hour
    minute = df[date_col].dt.minute

    combined = hour + minute/60

    df['hour_curve'] = np.sin(2*np.pi*combined/24)

    # Uses the date as index
    df.set_index(date_col, inplace=True)


In [51]:

# Change the date format and sets date as index
train_a.set_index('time', inplace=True)

convertDate(X_train_estimated_a, 'date_forecast')

convertDate(X_train_observed_a, 'date_forecast')

convertDate(X_test_estimated_a, 'date_forecast')

In [52]:
data = [X_train_estimated_a, X_train_observed_a, X_test_estimated_a]

for i in range(len(data)):
    print(type(data[i]))
    drop_col = [col for col in data[i].columns if 'snow' in col]
    drop_col.extend(['elevation:m', 'is_day:idx', 'clear_sky_energy_1h:J','dew_point_2m:K','diffuse_rad_1h:J','direct_rad_1h:J','pressure_100m:hPa','pressure_50m:hPa','sfc_pressure:hPa','t_1000hPa:K','total_cloud_cover:p', 'prob_rime:p', 'precip_type_5min:idx', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms', 'wind_speed_w_1000hPa:ms', 'dew_or_rime:idx', 'is_in_shadow:idx', 'super_cooled_liquid_water:kgm2'])
    data[i] = data[i].drop(columns=drop_col)

    data[i]['azimuth_cos_T'] = -np.cos(data[i]['sun_azimuth:d']/180*np.pi)
    data[i]['rad_x_cloudCover'] = data[i]['clear_sky_rad:W'] * data[i]['effective_cloud_cover:p']/100

    lead_features_df = data[i].copy()
    new_columns = []  # Create a list to store new columns
    new_column_names = []  # Create a list to store new column names

    for feature in data[i].columns.difference(['date_forecast', 'hour_curve', 'day_curve']):
        new_columns.append(data[i][feature].shift(1))
        new_columns.append(data[i][feature].shift(2))
        new_columns.append(data[i][feature].shift(3))
        new_columns.append(data[i][feature].shift(-1))
        new_columns.append(data[i][feature].shift(-2))
        new_columns.append(data[i][feature].shift(-3))

        new_column_names.extend([f'{feature}_{shift}m_lag' for shift in [15, 30, 45]])
        new_column_names.extend([f'{feature}_{shift}m_lead' for shift in [15, 30, 45]])
    
    # for feature in data[i].columns.difference(['date_forecast', 'hour_curve', 'day_curve']):
    #     new_columns.append(data[i][feature].shift(-1))
    #     new_columns.append(data[i][feature].shift(-2))
    #     new_columns.append(data[i][feature].shift(-3))
    #     new_column_names.extend([f'{feature}_{shift}m_lead' for shift in [15, 30, 45]])

    #     new_columns.append(data[i][feature].diff())
    #     new_column_names.extend([f'{feature}_diff'])

    # new_column_names = [f'{feature}_{shift}m_lag' for feature in data[i].columns.difference(['date_forecast', 'hour_curve', 'day_curve']) for shift in [15, 30, 45]]
    # new_column_names.extend([f'{feature}_{shift}m_lead' for feature in data[i].columns.difference(['date_forecast', 'hour_curve', 'day_curve']) for shift in [15, 30, 45]])
    new_columns = pd.concat(new_columns, axis=1)
    new_columns.columns = new_column_names

    data[i] = pd.concat([data[i], new_columns], axis=1)
    

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [53]:
X_train_estimated_a = data[0]
X_train_estimated_b = data[1]
X_train_estimated_c = data[2]
X_train_observed_a = data[3]
X_train_observed_b = data[4]
X_train_observed_c = data[5]
X_test_estimated_a = data[6]
X_test_estimated_b = data[7]
X_test_estimated_c = data[8]

# Preparing dataset

In [54]:
train_a_clean = pd.concat([X_train_observed_a, X_train_estimated_a]).join(train_a, how='inner')
train_a_clean = train_a_clean.dropna(subset=['pv_measurement'])

In [55]:
train_a_15 = pd.concat([X_train_observed_a, X_train_estimated_a]).join(train_a, how='outer')
train_a_15['pv_measurement'] = train_a_15['pv_measurement'].ffill()


### Removing bad data

A

In [56]:
start_date = '2022-10-21 00:00:00'
end_date = '2022-10-28 22:00:00'

train_a_clean.drop(train_a_clean.loc[start_date:end_date].index, inplace=True)
train_a_15.drop(train_a_15.loc[start_date:end_date].index, inplace=True)

### Splitting to X and y

In [59]:
X_train_a_clean = train_a_clean.drop(columns=['pv_measurement'])
y_train_a_clean = train_a_clean['pv_measurement']

X_test_a_clean = X_test_estimated_a

X_train_a_15 = train_a_15.drop(columns=['pv_measurement'])


Validation data

In [60]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error,\
  mean_squared_error

def evaluate_model(y_test, prediction):
  print(f"MAE: {mean_absolute_error(y_test, prediction)}")
  print(f"MSE: {mean_squared_error(y_test, prediction)}")
  print(f"MAPE: {mean_absolute_percentage_error(y_test, prediction)}")


In [61]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the data and transform the data
scaler.fit(X_train_a_clean)
X_train_a_clean_scaled = scaler.transform(X_train_a_clean)
X_test_a_clean_scaled = scaler.transform(X_test_a_clean)
X_train_a_15_scaled = scaler.transform(X_train_a_15)


# Convert the scaled data back into a DataFrame
X_train_a_clean = pd.DataFrame(X_train_a_clean_scaled, columns=X_train_a_clean.columns, index=X_train_a_clean.index)

X_test_a_clean = pd.DataFrame(X_test_a_clean_scaled, columns=X_test_a_clean.columns, index=X_test_a_clean.index)

X_train_a_15 = pd.DataFrame(X_train_a_15_scaled, columns=X_train_a_15.columns, index=X_train_a_15.index)

In [62]:
from sklearn.model_selection import train_test_split
import re
X_train_a_clean = X_train_a_clean.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

X_test_a_clean = X_test_a_clean.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

X_train_a_15 = X_train_a_15.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

X_train_a, X_val_a, y_train_a, y_val_a = train_test_split(X_train_a_clean, y_train_a_clean, test_size=0.1, random_state=42)

# Defining Stacking functions

In [63]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb

def train_and_predict(models, X, y, X_test, n_splits=5):
    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)

    # Initialize a dictionary to hold the predictions of each model
    model_train_predictions = {}
    model_test_predictions = {}

    # Iterate over each model
    for model_name, model in models.items():
        print(f'Training {model_name}...')
        
        # Initialize a DataFrame to hold the predictions of the current model
        model_train_predictions[model_name] = pd.DataFrame()
        model_test_predictions[model_name] = pd.DataFrame()

        # K-fold Cross Validation model evaluation
        fold_no = 1
        for train, test in kfold.split(X, y):
            # Generate a print
            print('------------------------------------------------------------------------')
            print(f'Training for fold {fold_no} ...')

            # Fit data to model
            model.fit(X.iloc[train], y.iloc[train])

            # Generate generalization metrics
            predictions = model.predict(X.iloc[test])
            print(f'Score for fold {fold_no}: {mean_absolute_error(y.iloc[test], predictions)}')

            # Store the predictions of the current fold in a DataFrame
            fold_predictions = pd.DataFrame(model.predict(X))
            model_train_predictions[model_name] = pd.concat([model_train_predictions[model_name], fold_predictions], axis=1)

            # Predict on the test set and store the predictions in a DataFrame
            test_predictions = pd.DataFrame(model.predict(X_test))
            model_test_predictions[model_name] = pd.concat([model_test_predictions[model_name], test_predictions], axis=1)

            fold_no = fold_no + 1

        # Sort the train set predictions and average the test set predictions across all folds
        model_train_predictions[model_name] = model_train_predictions[model_name].mean(axis=1)
        model_test_predictions[model_name] = model_test_predictions[model_name].mean(axis=1)

    return model_train_predictions, model_test_predictions



# Define your models
models = {
    'lgbm_model': lgb.LGBMRegressor(
        n_jobs=4,
        colsample_bytree = 0.9475337647224076, 
        learning_rate = 0.045592564258931566, 
        max_depth = 8, 
        n_estimators = 700, 
        num_leaves = 100, 
        subsample = 0.601967016806479
    ),
    'cat_model': CatBoostRegressor(
        bagging_temperature = 0.7378554663350414,
        border_count = 100, 
        depth = 8, 
        iterations = 1000, 
        l2_leaf_reg = 0.012184856499092163, 
        learning_rate = 0.08771473097605541, 
        random_strength = 0.5010746002603824,
        eval_metric='MAE'
    ),
    'xgb_model' : xgb.XGBRegressor(
        colsample_bylevel = 0.6271084032736475,
        colsample_bytree = 0.6891200569300842,
        gamma = 0.7188221908641431,
        iterations = 1000,
        learning_rate = 0.07066060748279693,
        max_depth = 8,
        min_child_weight = 3,
        n_estimators = 200,
        reg_alpha = 1.0561019503845397,
        reg_lambda = 4.886536933525814,
        subsample = 0.9834397418107923
    )
}

In [64]:
stacking_model = {
    'XGBregressor': xgb.XGBRegressor(
    colsample_bylevel = 0.6625351881046275, 
    colsample_bytree = 0.7090615971304609, 
    gamma = 0.30325853173466527, 
    iterations = 400, 
    learning_rate = 0.04329055231549767, 
    max_depth = 5, 
    min_child_weight = 6, 
    n_estimators = 170, 
    reg_alpha = 0.003769836675097778, 
    reg_lambda = 2.48928058741362, 
    subsample = 0.500100229092168,
    eval_metric='mae'
    )
}

# Remove Outliers

In [65]:
from sklearn.model_selection import KFold
import lightgbm as lgb

def find_outliers(X_data, y_data, model, threshold, n_splits=5):
    # Initialize the KFold object
    kfold = KFold(n_splits=n_splits)
    
    # Separate features and target
    X = X_data
    y = y_data
    
    # Placeholder for outliers
    outliers = []
    
    for train_index, test_index in kfold.split(X):
        # Split the data into training and testing sets
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions on the test set
        predictions = model.predict(X_test)
        
        # Find instances where the absolute difference between 
        # the predicted and actual values is greater than the threshold
        diff = np.abs(predictions - y_test)
        outlier_indices = diff[diff > threshold].index.tolist()
        
        # Add these outliers to the list
        outliers.extend(outlier_indices)
    
    # Return a dataframe of just the outliers
    return X_data.loc[outliers]



model = lgb.LGBMRegressor(
        n_jobs=4,
        colsample_bytree = 0.9475337647224076, 
        learning_rate = 0.045592564258931566, 
        max_depth = 8, 
        n_estimators = 700, 
        num_leaves = 100, 
        subsample = 0.601967016806479
    )
    
threshold_a = 2500
outliers_a = find_outliers(X_train_a_clean, y_train_a_clean, model, threshold_a)
print(f"Outliers found in A with threshold of {threshold_a}: {len(outliers_a)}")


Outliers found in A with threshold of 2500: 107
Outliers found in B with threshold of 500: 56
Outliers found in C with threshold of 400: 57


In [66]:
X_train_a_clean.drop(outliers_a.index, axis=0, inplace=True)
y_train_a_clean.drop(outliers_a.index, axis=0, inplace=True)


# Stacking with removing outliers

In [67]:
X_train_a_stacked, X_test_a_stacked = train_and_predict(models, X_train_a_clean, y_train_a_clean, X_test_a_clean)

Training lgbm_model...
------------------------------------------------------------------------
Training for fold 1 ...
Score for fold 1: 170.4823948938226
------------------------------------------------------------------------
Training for fold 2 ...
Score for fold 2: 168.25586592449736
------------------------------------------------------------------------
Training for fold 3 ...
Score for fold 3: 163.59509676253094
------------------------------------------------------------------------
Training for fold 4 ...
Score for fold 4: 166.25720739219432
------------------------------------------------------------------------
Training for fold 5 ...
Score for fold 5: 165.96619494313452
Training cat_model...
------------------------------------------------------------------------
Training for fold 1 ...
0:	learn: 765.8717293	total: 13.1ms	remaining: 13.1s
1:	learn: 707.8350541	total: 25.5ms	remaining: 12.7s
2:	learn: 656.2304879	total: 37.9ms	remaining: 12.6s
3:	learn: 609.4303107	total: 4

In [68]:
X_train_a_stacked = pd.DataFrame(X_train_a_stacked)
X_test_a_stacked = pd.DataFrame(X_test_a_stacked)

## Extending dataset with features

In [69]:
X_train_a_stacked.index = X_train_a_clean.index
X_test_a_stacked.index = X_test_a_clean.index

X_train_a_stacked_extended = pd.concat([X_train_a_stacked, X_train_a_clean[['rad_x_cloudCover']]], axis=1)
X_test_a_stacked_extended = pd.concat([X_test_a_stacked, X_test_a_clean[['rad_x_cloudCover']]], axis=1)

In [70]:
_, pred_a_nooutliers_radx = train_and_predict(stacking_model, X_train_a_stacked_extended, y_train_a_clean, X_test_a_stacked_extended, n_splits=5)

Training XGBregressor...
------------------------------------------------------------------------
Training for fold 1 ...
[14:04:34] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "iterations" } are not used.

Score for fold 1: 64.79669166508457
------------------------------------------------------------------------
Training for fold 2 ...
[14:04:34] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "iterations" } are not used.

Score for fold 2: 64.97274173371646
------------------------------------------------------------------------
Training for fold 3 ...
[14:04:35] WARNING: C:\Users\dev-admin\croot2\xgboost-split_1675461376218\work\src\learner.cc:767: 
Parameters: { "iterations" } are not used.

Score for fold 3: 63.112457575934
------------------------------------------------------------------------
Training for fold 4 ...
[14:04:36] WARNING: C:\Users\dev-admin\croot2\xgb

In [71]:
pred_a_nooutliers_radx = pred_a_nooutliers_radx['XGBregressor']


# Results

In [72]:
def preprocess_data(pred):
    pred_stacked = pd.DataFrame(pred, columns=['pv_measurement'])
    pred_stacked = pred_stacked.iloc[::4]
    pred_stacked = np.array(pred_stacked)
    return pred_stacked

In [73]:
pred_a_nooutliers_radx_processsed = preprocess_data(pred_a_nooutliers_radx)


In [74]:
X_test_a_red = is_night_data.iloc[::4]
condition = X_test_a_red['is_night'] == 1.0
condition.reset_index(drop=True, inplace=True)
pred_a_nooutliers_radx_processsed[condition] = 0.0
